<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/Practica1/3_2_scrapyJSON_LD_LaOpinion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3 - Scrapy - Ejemplo Crawling JSON-LD (ElMundo)

## Apartado 1.1 Crawler de El Mundo extrayendo JSON-LD
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
El Mundo y otros periódicos publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org

** Existen librerías de Python para trabajar directamente con JSON-LD

In [1]:
!pip3 install -U scrapy

In [2]:
import scrapy
import sys
import json
import locale
import time
import random
import hashlib
import re

from bs4 import BeautifulSoup

class LaOpinionSpider (scrapy.Spider):

    #Es obligatorio poner un nombre
    name = 'laOpinion'

    #Ponemos que el dominio que está permitido es el de la página y no vamos a irnos fuera de la misma
    allowed_domains = ['https://www.laopiniondemurcia.es']

    start_urls = ['https://www.laopiniondemurcia.es/deportes/2023/12/01/guerra-gaza-deporte-veto-hipocrita-95328986.html']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    nDocumentos = 0
    visited_urls = set()
    def parse (self, response):
      """
      @inherit

      @param self
      @param response
      """

      url = str(response.request.url).strip()
      #Buscamos todos los elementos en el archivo XML con la etiqueta <item>
      for item in response.css ('.bbnx-template.article-template'):

            #Obtenemos por cada elemento <item> el texto del subelemento <title>. Además co el BeautifulSoup
            #procesamos el texto en html y nos quedamos con el texto
            title = BeautifulSoup(str(item.css ('.h1::text').get()), 'html.parser').get_text().strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <h2>
            subtitle = BeautifulSoup(str(item.css ('.headline-article__extended-subtitle').get()), 'html.parser').get_text().strip()

            author = BeautifulSoup(str(item.css('.news-author').get()), 'html.parser').get_text().strip()

            date = BeautifulSoup(str(item.css(".article-author__date").get()), "html.parser").get_text().strip()

            content = BeautifulSoup(str(item.css ('.article-body ').get()), 'html.parser').get_text().strip()

            #Imprimimos la información obtenida para comprobar lo que estamos extrayendo
            print ("-------------------------")
            print ('URL:' + url)
            print ("Date : "+date)
            print ('Author:' + author)
            print ('Título:' + title)
            print ('Subtitulo:' + subtitle)
            print ('Contenido:' + content)
            print ("-------------------------")

            data = {
                'url' : url,
                "date": date,
                'author': author,
                'title': title,
                'subtitle':subtitle,
                'content': content,
            }

            title_hash = hashlib.md5(title.encode()).hexdigest()

            filename = f"{title_hash}.json"

            with open ('laOpinion/' + filename, 'w') as f:
                 json.dump (data, f, indent = 4)

            url_in_current_document = response.css ('a')

            print(url_in_current_document)
            for next_page in url_in_current_document:
              # Para limitar que solamente se parseen las noticias dentro de 'https://www.um.es/web/sala-prensa/'
              # obtenemos el atributo href de la etiqueta <a> y parseamos la página

              url_str = str(next_page.css('::attr(href)').get())
              print("PÁGINA: ", url_str)

              regex = r"^(https://www.laopiniondemurcia.es/(deportes|nacional|murcia|ocio))"
              if re.match(regex, url_str) and url_str not in self.visited_urls and self.nDocumentos < 1500:
                print("PÁGINA VALIDA: ", url_str)
                self.nDocumentos = self.nDocumentos + 1
                self.visited_urls.add(url_str)
                yield response.follow (next_page, self.parse)




## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

# Nueva sección

In [3]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('laOpinion')):
    os.mkdir('laOpinion')

# Creamos el proceso con el RSSSpider
process.crawl(LaOpinionSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
URL:https://www.laopiniondemurcia.es/deportes/2023/12/01/guerra-gaza-deporte-veto-hipocrita-95328986.html
Date : 01·12·23
Author:Denís Iglesias
Título:La guerra de Gaza en el deporte: el veto "hipócrita" de las banderas de Palestina y el peregrinaje de Israel
Subtitulo:Jugadores y aficionados han sido sancionados por mostrar su apoyo al pueblo palestino y pedir el alto al fuego de Israel, cuyos equipos juegan a puerta cerrada en competiciones europeas y fuera de su país
Contenido:El conflicto entre Palestina e Israel también es motivo de conflicto en el deporte. Los palestinos denuncian el veto de sus banderas, que han utilizado diferentes aficionados y jugadores para mostrarles su apoyo. Por su parte, los israelís critican el peregrinaje en el exilio que están haciendo en competiciones europeas. Para entender este cisma, el doble partido disputado entre el Breogán español y el Hapoel Holon de Israel en la Basketball Champions League, competición organizada po

INFO:scrapy.core.engine:Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 364,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 142305,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 0.753975,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 12, 1, 11, 22, 37, 10426, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 261612,
 'httpcompression/response_count': 1,
 'log_count/DEBUG': 3,
 'log_count/INFO': 10,
 'memusage/max': 141053952,
 'memusage/startup': 141053952,
 'offsite/domains': 1,
 'offsite/filtered': 20,
 'request_depth_max': 1,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2023, 12, 1, 11, 22, 36, 256451, tzinfo=datetime.timezone.utc)}
INFO:scr

PÁGINA:  https://www.laopiniondemurcia.es/salud/
PÁGINA:  https://www.laopiniondemurcia.es/vida-y-estilo/tecnologia/
PÁGINA:  https://www.laopiniondemurcia.es/vida-y-estilo/moda-belleza/
PÁGINA:  https://www.laopiniondemurcia.es/vida-y-estilo/gente/
PÁGINA:  https://www.laopiniondemurcia.es/vida-y-estilo/decoracion/
PÁGINA:  https://www.laopiniondemurcia.es/buzzeando/
PÁGINA:  https://www.laopiniondemurcia.es/vida-y-estilo/mascotas/
PÁGINA:  https://www.laopiniondemurcia.es/motor/
PÁGINA:  https://www.compramejor.es/
PÁGINA:  https://www.laopiniondemurcia.es/buscando-respuestas/
PÁGINA:  https://www.laopiniondemurcia.es/videos/
PÁGINA:  https://www.laopiniondemurcia.es/fotos/
PÁGINA:  https://www.iberempleos.es/ofertas-empleo/murcia
PÁGINA:  https://www.tucasa.com/compra-venta/viviendas/murcia/?r=&idz=0030
PÁGINA:  https://www.tucasa.com/alquiler/viviendas/murcia/?r=&idz=0030
PÁGINA:  https://ocasion.neomotor.com/coches-segunda-mano-murcia
PÁGINA:  https://www.laopiniondemurcia.es/blog